In [1]:
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import os
import re # Import regular expression module

# --- Configuration ---
h5ad_file_path = "./merged_spatial_h5ad/merged_spatial_rois_RAW.h5ad"  # <--- CHANGE THIS to your input file path
output_h5ad_path = "./merged_spatial_h5ad/filtered_merged_spatial_rois_RAW.h5ad" # <--- CHANGE THIS to your desired output file path

# Gene lists/patterns to remove
MITO_PREFIX = 'mt-' # Mouse convention, use 'MT-' for human if needed
RPS_PATTERN = r'^Rp[sl]' # Regular expression for ribosomal protein genes (Rp + s or l)
HSP_PREFIX = 'Hsp'     # Prefix for heat shock protein genes
DISSOCIATION_GENES = [
    'Atf3','Zfp36','Fos','Dusp1','Gadd45g','Jun','Junb','Jund',
    'Actg1','Ier2','Sat1','Ubc','Nr4a1','Fosb','Ppp1r15a', # Removed duplicate Fos
    'Mt1','Mt2','Errfi1','Ier3','Nfkbia','Zfp36l1','Cebpb','Cebpd','Btg2'
]

# Files to save removed gene lists (optional)
SAVE_GENE_LISTS = True # Set to False if you don't need these files
mito_gene_file = "Mitochondria_genes.csv"
ribo_gene_file = "Ribosome_genes.csv"
hsp_gene_file = "Heat_shock_protein_genes.csv"
diss_gene_file = "Dissociation-induced_genes.csv"
# --- End Configuration ---

print(f"Loading AnnData object from: {h5ad_file_path}")
adata = ad.read_h5ad(h5ad_file_path)
adata.var_names_make_unique()
print(f"Original AnnData object shape: {adata.shape}") # (n_obs, n_vars) -> (spots/cells, genes)

# Ensure var_names are strings for filtering
adata.var_names = adata.var_names.astype(str)

# --- Identify genes to remove ---

# 1. Mitochondrial genes
is_mito = adata.var_names.str.startswith(MITO_PREFIX)
mito_genes = adata.var_names[is_mito].tolist()
print(f"Found {len(mito_genes)} mitochondrial genes starting with '{MITO_PREFIX}'.")
if SAVE_GENE_LISTS and mito_genes:
    pd.Series(mito_genes).to_csv(mito_gene_file, index=False, header=False)
    print(f"Saved mitochondrial gene list to {mito_gene_file}")

# 2. Ribosomal protein genes
# Use re.match for each gene name via a list comprehension or apply
is_ribo = np.array([bool(re.match(RPS_PATTERN, gene, re.IGNORECASE)) for gene in adata.var_names])
# Alternatively using pandas str.match (can be faster):
# is_ribo = adata.var_names.str.match(RPS_PATTERN, case=False) # Use case=False if needed

ribo_genes = adata.var_names[is_ribo].tolist()
print(f"Found {len(ribo_genes)} ribosomal genes matching pattern '{RPS_PATTERN}'.")
if SAVE_GENE_LISTS and ribo_genes:
    pd.Series(ribo_genes).to_csv(ribo_gene_file, index=False, header=False)
    print(f"Saved ribosomal gene list to {ribo_gene_file}")

# 3. Heat shock protein genes
is_hsp = adata.var_names.str.startswith(HSP_PREFIX)
hsp_genes = adata.var_names[is_hsp].tolist()
print(f"Found {len(hsp_genes)} heat shock protein genes starting with '{HSP_PREFIX}'.")
if SAVE_GENE_LISTS and hsp_genes:
    pd.Series(hsp_genes).to_csv(hsp_gene_file, index=False, header=False)
    print(f"Saved heat shock protein gene list to {hsp_gene_file}")

# 4. Dissociation-induced genes
# Ensure the list contains only genes actually present in the data
diss_genes_in_data = [gene for gene in DISSOCIATION_GENES if gene in adata.var_names]
is_diss = adata.var_names.isin(diss_genes_in_data)
print(f"Found {len(diss_genes_in_data)} dissociation-induced genes from the provided list present in the data.")
if SAVE_GENE_LISTS and diss_genes_in_data:
    pd.Series(diss_genes_in_data).to_csv(diss_gene_file, index=False, header=False)
    print(f"Saved dissociation-induced gene list to {diss_gene_file}")

# --- Combine filters and subset AnnData ---

# Create a combined boolean mask for ALL genes to remove
genes_to_remove_mask = is_mito | is_ribo | is_hsp | is_diss

# Create a mask for genes to KEEP (invert the remove mask)
genes_to_keep_mask = ~genes_to_remove_mask

n_genes_to_remove = genes_to_remove_mask.sum()
n_genes_to_keep = genes_to_keep_mask.sum()

print(f"\nTotal genes to remove: {n_genes_to_remove}")
print(f"Total genes to keep: {n_genes_to_keep}")

if n_genes_to_keep == 0:
     print("Warning: All genes were marked for removal. Check your patterns/lists.")
elif n_genes_to_keep == adata.n_vars:
     print("No genes matched the removal criteria.")
else:
    # Subset the AnnData object, keeping only the desired genes
    # Use .copy() to create a new object in memory, not just a view
    adata_filtered = adata[:, genes_to_keep_mask].copy()

    print(f"\nFiltered AnnData object shape: {adata_filtered.shape}")

    # --- Save the filtered AnnData object ---
    print(f"Saving filtered AnnData object to: {output_h5ad_path}")
    adata_filtered.write_h5ad(output_h5ad_path)
    print("Filtering complete.")

Loading AnnData object from: ./merged_spatial_h5ad/merged_spatial_rois_RAW.h5ad
Original AnnData object shape: (4991, 19059)
Found 13 mitochondrial genes starting with 'mt-'.
Saved mitochondrial gene list to Mitochondria_genes.csv
Found 0 ribosomal genes matching pattern '^Rp[sl]'.
Found 30 heat shock protein genes starting with 'Hsp'.
Saved heat shock protein gene list to Heat_shock_protein_genes.csv
Found 23 dissociation-induced genes from the provided list present in the data.
Saved dissociation-induced gene list to Dissociation-induced_genes.csv

Total genes to remove: 66
Total genes to keep: 18993

Filtered AnnData object shape: (4991, 18993)
Saving filtered AnnData object to: ./merged_spatial_h5ad/filtered_merged_spatial_rois_RAW.h5ad
Filtering complete.


In [2]:
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np

adata = ad.read_h5ad('./merged_spatial_h5ad/filtered_merged_spatial_rois_RAW.h5ad')
adata.var_names_make_unique()
adata.layers["counts"] = adata.X.copy()

In [3]:
import pandas as pd
import scipy.sparse as sp
import anndata as ad # Assuming adata is an AnnData object

# --- Assume 'adata' is your loaded AnnData object ---
# Example:
# adata = ad.read_h5ad("your_anndata_file.h5ad")

# --- Define the layer key and output filename ---
layer_key = 'counts'
output_csv_file = './e14_e15_e16_raw_counts.csv'

# --- Check if the layer exists ---
if layer_key not in adata.layers:
    print(f"Error: Layer '{layer_key}' not found in adata.layers.")
else:
    print(f"Accessing layer: '{layer_key}'")
    # Get the data from the specified layer
    layer_data = adata.layers[layer_key]

    # Convert to dense NumPy array (handles both sparse and dense input)
    # Warning: This can use a lot of memory if the layer is large and sparse!
    print("Converting layer data to dense NumPy array...")
    try:
        dense_array = layer_data.toarray() if sp.issparse(layer_data) else layer_data
        print("Conversion successful.")

        # Create the pandas DataFrame
        print("Creating pandas DataFrame...")
        counts_df = pd.DataFrame(
            data=dense_array,
            index=adata.obs_names,
            columns=adata.var_names
        )
        print("DataFrame created.")

        # Save the DataFrame to CSV
        print(f"Saving DataFrame to '{output_csv_file}'...")
        counts_df.to_csv(output_csv_file)
        print("Save complete.")

    except MemoryError:
        print("\nError: Ran out of memory trying to convert the layer to a dense array or create the DataFrame.")
        print("The 'ReX' layer is likely too large to be processed this way.")
        print("Consider saving as a sparse format (like MTX) or processing in chunks if possible.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

Accessing layer: 'counts'
Converting layer data to dense NumPy array...
Conversion successful.
Creating pandas DataFrame...
DataFrame created.
Saving DataFrame to './e14_e15_e16_raw_counts.csv'...
Save complete.


In [4]:
spatial_coords = adata.obsm['spatial']
spatial_coords_df = pd.DataFrame(spatial_coords, columns=['x', 'y'], index=adata.obs_names)
spatial_coords_df.to_csv('./e14_e15_e16_coordinates.csv')

In [1]:
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import os
import re # Import regular expression module

In [2]:
adata = sc.read('./merged_spatial_h5ad/merged_spatial_rois_RAW.h5ad')
adata.var_names_make_unique

<bound method AnnData.var_names_make_unique of AnnData object with n_obs × n_vars = 4991 × 19059
    obs: 'id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'geometry_wkt', 'area', 'tissue_section_id', 'sample_id'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'spatial'
    obsm: 'spatial'>

In [3]:
cluster_df = pd.read_csv('./filtered_merged_spatial_rois_RAW_metadata.csv')
cluster_df

,Unnamed: 0,barcodes,sample_id,BASS_clusters
0,1,e14_ID_30818,e14,Resting chondrocytes
1,2,e14_ID_30820,e14,Resting chondrocytes
2,3,e14_ID_30829,e14,Resting chondrocytes
3,4,e14_ID_30831,e14,Resting chondrocytes
4,5,e14_ID_30832,e14,Proliferative chondrocytes
...,...,...,...,...
4986,4987,e16_ID_60488,e16,Prehypertrophic chondrocytes
4987,4988,e16_ID_60494,e16,Tendon sheath progenitors
4988,4989,e16_ID_60495,e16,Prehypertrophic chondrocytes
4989,4990,e16_ID_60500,e16,Prehypertrophic chondrocytes


In [2]:
import anndata as ad
import pandas as pd
import numpy as np

# --- Configuration ---
h5ad_file = './merged_spatial_h5ad/filtered_merged_spatial_rois_RAW.h5ad'  # Path to your H5AD file
# V V V V  MAKE SURE THIS PATH IS CORRECT V V V V
cluster_file = './filtered_merged_spatial_rois_RAW_metadata.csv'
# V V V V  CHANGE THIS TO MATCH YOUR CSV V V V V
cell_id_column = 'barcodes'            # Column name in CSV with cell IDs/barcodes
# V V V V  MAKE SURE THIS MATCHES YOUR CSV V V V V
cluster_column_name_in_csv = 'BASS_clusters' # Column name in CSV with cluster labels
output_h5ad_file = './merged_spatial_h5ad/BASS_clustered_filtered_merged_spatial_rois_RAW.h5ad' # Name for the output file
adata_obs_cluster_column = 'BASS_clusters'  # Name for the new column in adata.obs

# --- Step 1: Load the existing AnnData object ---
try:
    adata = ad.read_h5ad(h5ad_file)
    print(f"Loaded AnnData object: {adata}")
    # Optional: Check if adata.obs_names look like the 'barcodes' column
    print("\nFirst 5 AnnData obs_names:")
    print(adata.obs_names[:5])
except FileNotFoundError:
    print(f"Error: H5AD file not found at '{h5ad_file}'")
    # Handle error or exit if necessary
    exit() # Or create dummy data if that's intended

# --- Step 2: Load the cell-level cluster assignments ---
try:
    cluster_df = pd.read_csv(cluster_file)
    print(f"\nLoaded cluster data from '{cluster_file}':")
    print("First 5 rows of cluster_df:")
    print(cluster_df.head())
    print("\nColumns found:", list(cluster_df.columns))

    # --- Step 3: Align cluster data with AnnData object ---
    # Check if the necessary columns exist before proceeding
    if cell_id_column not in cluster_df.columns:
        raise KeyError(f"Column '{cell_id_column}' not found in {cluster_file}. Available columns: {list(cluster_df.columns)}")
    if cluster_column_name_in_csv not in cluster_df.columns:
        raise KeyError(f"Column '{cluster_column_name_in_csv}' not found in {cluster_file}. Available columns: {list(cluster_df.columns)}")

    # Set the correct cell ID column as the index for easy alignment
    cluster_df = cluster_df.set_index(cell_id_column) # <-- Uses 'barcodes' now

    # Ensure all cells in adata are present in the cluster file
    if not all(cell in cluster_df.index for cell in adata.obs_names):
        missing_cells = adata.obs_names.difference(cluster_df.index)
        print(f"\nWarning: {len(missing_cells)} cells in AnnData object are missing from cluster file '{cluster_file}'.")
        print(f"First few missing: {list(missing_cells[:5])}")
        # Decide how to handle missing cells (reindex and fill, error, etc.)
        # Option: Reindex and fill missing with a placeholder like 'Unknown'
        cluster_assignments = cluster_df.reindex(adata.obs_names)[cluster_column_name_in_csv]
        cluster_assignments = cluster_assignments.fillna('Unknown')
        print("Missing cells in cluster file filled with 'Unknown' in the final assignments.")
        # Option: Raise an error if alignment must be perfect
        # raise ValueError(f"Mismatch: {len(missing_cells)} cells missing in cluster file.")
    else:
        # Reindex the cluster data to match the exact order of cells in adata.obs_names
        cluster_assignments = cluster_df.loc[adata.obs_names, cluster_column_name_in_csv]

    # Verify length
    if len(cluster_assignments) != adata.n_obs:
        raise ValueError(f"Length mismatch after alignment! AnnData obs: {adata.n_obs}, Cluster assignments: {len(cluster_assignments)}")

except FileNotFoundError:
    print(f"\nError: Cluster file not found at '{cluster_file}'.")
    exit() # Stop execution
except KeyError as e:
    print(f"\nError processing cluster file: {e}")
    exit() # Stop execution
except Exception as e:
    print(f"\nAn unexpected error occurred during cluster loading/alignment: {e}")
    exit() # Stop execution


# --- Step 4: Add the cluster assignments to adata.obs ---
adata.obs[adata_obs_cluster_column] = cluster_assignments
adata.obs[adata_obs_cluster_column] = adata.obs[adata_obs_cluster_column].astype('category')

print(f"\nAdded '{adata_obs_cluster_column}' column to adata.obs.")
print("Value counts in the new column:")
# Calculate and print value counts, handling potential 'Unknown' category
print(adata.obs[adata_obs_cluster_column].value_counts(dropna=False))


# --- Step 5: Save the updated AnnData object ---
try:
    adata.write_h5ad(output_h5ad_file)
    print(f"\nSuccessfully saved updated AnnData object to '{output_h5ad_file}'")
except Exception as e:
    print(f"\nError saving H5AD file: {e}")

Loaded AnnData object: AnnData object with n_obs × n_vars = 4991 × 18993
    obs: 'id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'geometry_wkt', 'area', 'tissue_section_id', 'sample_id'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'spatial'
    obsm: 'spatial'

First 5 AnnData obs_names:
Index(['e14_ID_30818', 'e14_ID_30820', 'e14_ID_30829', 'e14_ID_30831',
       'e14_ID_30832'],
      dtype='object')

Loaded cluster data from './filtered_merged_spatial_rois_RAW_metadata.csv':
First 5 rows of cluster_df:
   Unnamed: 0      barcodes sample_id               BASS_clusters
0           1  e14_ID_30818       e14        Resting chondrocytes
1           2  e14_ID_30820       e14        Resting chond